In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [2]:
movies = pd.read_csv('movies_and_posters.csv', sep=';')
movies.head()

,Unnamed: 0,movieId,title,genres,imdbId,tmdbId,poster_path
0,0,2256,Parasite (1982),Horror|Sci-Fi,84472,48311,2256.jpg
1,1,6530,"Tenant, The (Locataire, Le) (1976)",Drama|Horror|Mystery|Thriller,74811,11482,6530.jpg
2,2,52202,His Kind of Woman (1951),Comedy|Crime|Drama|Film-Noir|Thriller,43643,33673,52202.jpg
3,3,61868,Jam (2006),Drama,482528,71139,61868.jpg
4,4,74297,Electric Shadows (Meng ying tong nian) (2004),Drama,424273,26137,74297.jpg


In [ ]:
X_train, X_test = train_test_split(movies, test_size=0.1, random_state=1)

In [ ]:
batch_size = 20
img_width, img_height = 224, 224
#img_width, img_height = 299, 299

def batch_generator(dataframe, distortion=False):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=False
    )   

    row_generator = dataframe.sample(frac=1).iterrows()
    
    for batch_count in range(int(dataframe.shape[0]/batch_size)):
        
        batch_img = []
        batch_img_files = []
        batch_labels = []
        batch_fgs = []
        batch_density = []
        
        for img_count in range(batch_size):
            _, row = next(row_generator)

            img = load_img(os.path.join(master_image_dir, row[existing_path_col]),
                         target_size=(img_width, img_height))
            img = img_to_array(img)
            if hasattr(img, 'close'):
                img.close()
            if distortion:
                img = train_datagen.random_transform(img)
            img = train_datagen.standardize(img)

            label_list = row[class_col_id].split('|')
            label = label_binarizer.transform([row[class_col_id]])[0]

            batch_img.append(img)
            batch_img_files.append(row[existing_path_col])
            batch_labels.append(label)
            batch_fgs.append(fg_class_distributions[row['mainProductRangeId']])
            batch_density.append(row['density_normalized'])

        yield [np.array(batch_img), np.array(batch_fgs), np.array(batch_density)], np.array(batch_labels), [batch_img_files]
            
    return